# Funciones Reutilizables del Proyecto

Este notebook contiene funciones útiles que pueden ser reutilizadas en cualquier parte del proyecto de análisis de energía renovable.

## Contenido:
1. **Carga y preprocesamiento de datos**
2. **Visualización**
3. **Métricas de evaluación**
4. **Utilidades generales**


## 1. Importaciones


In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10


## 2. Funciones de Carga y Preprocesamiento de Datos


In [ ]:
def load_capacidad_data(file_path, sep=';', decimal=','):
    """
    Carga el archivo CSV de capacidad acumulada con el formato específico del proyecto.
    
    Parameters:
    -----------
    file_path : str
        Ruta al archivo CSV
    sep : str
        Delimitador (default: ';')
    decimal : str
        Separador decimal (default: ',')
    
    Returns:
    --------
    pd.DataFrame
        DataFrame con los datos cargados y procesados
    """
    df = pd.read_csv(file_path, sep=sep, decimal=decimal, encoding='utf-8-sig')
    
    # Convertir la columna Category a datetime
    df['Category'] = pd.to_datetime(df['Category'], format='%Y-%m-%d')
    
    # Ordenar por fecha
    df = df.sort_values('Category').reset_index(drop=True)
    
    return df

def load_capacidad_csv_data(file_path, sep=';', decimal=','):
    """
    Carga el archivo CSV de capacidad acumulada con el formato específico del proyecto.
    
    Parameters:
    -----------
    file_path : str
        Ruta al archivo CSV
    sep : str
        Delimitador (default: ';')
    decimal : str
        Separador decimal (default: ',')
    
    Returns:
    --------
    pd.DataFrame
        DataFrame con los datos cargados y procesados
    """
    df = pd.read_csv(file_path, sep=sep, decimal=decimal, encoding='utf-8-sig')
    
    return df

def preprocess_capacidad_data(df):
    """
    Preprocesa los datos de capacidad acumulada.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame con los datos de capacidad
    
    Returns:
    --------
    pd.DataFrame
        DataFrame preprocesado
    """
    df_processed = df.copy()
    
    # Crear columnas de tiempo
    df_processed['Year'] = df_processed['Category'].dt.year
    df_processed['Month'] = df_processed['Category'].dt.month
    df_processed['Day'] = df_processed['Category'].dt.day
    df_processed['DayOfYear'] = df_processed['Category'].dt.dayofyear
    
    return df_processed

def extract_first_day_per_month(df, date_column='Category', verbose=True):
    """
    Extrae el primer día disponible de cada mes desde un DataFrame de datos temporales.
    
    Esta función procesa un DataFrame con datos temporales y retorna un nuevo DataFrame que contiene
    solo el primer día disponible de cada mes para todos los años, reduciendo significativamente
    la cantidad de datos a procesar.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame con datos temporales. Debe contener una columna de fechas.
    date_column : str
        Nombre de la columna que contiene las fechas (default: 'Category').
        La columna debe ser de tipo datetime o convertible a datetime.
    verbose : bool
        Si es True, imprime información del proceso (default: True)
    
    Returns:
    --------
    pd.DataFrame
        DataFrame con solo el primer día disponible de cada mes, ordenado por fecha
    
    Example:
    --------
    >>> # Cargar datos
    >>> df = load_capacidad_data('capacidad_acumulada.csv')
    >>> 
    >>> # Extraer primer día de cada mes
    >>> df_reduced = extract_first_day_per_month(df, date_column='Category')
    >>> 
    >>> # Ver resultados
    >>> print(f"Registros originales: {len(df)}")
    >>> print(f"Registros reducidos: {len(df_reduced)}")
    """
    # Hacer una copia para no modificar el DataFrame original
    df_work = df.copy()
    
    # Asegurar que la columna de fecha sea datetime
    if not pd.api.types.is_datetime64_any_dtype(df_work[date_column]):
        df_work[date_column] = pd.to_datetime(df_work[date_column], errors='coerce')
    
    # Filtrar filas con fechas válidas
    df_work = df_work.dropna(subset=[date_column])
    
    if len(df_work) == 0:
        if verbose:
            print("⚠ Advertencia: No se encontraron fechas válidas en el DataFrame")
        return pd.DataFrame()
    
    # Crear columnas auxiliares de año y mes
    df_work['_Year'] = df_work[date_column].dt.year
    df_work['_Month'] = df_work[date_column].dt.month
    
    # Agrupar por año y mes, y tomar el primer día (mínimo) de cada grupo
    first_day_per_month = df_work.groupby(['_Year', '_Month']).apply(
        lambda x: x.loc[x[date_column].idxmin()]
    ).reset_index(drop=True)
    
    # Eliminar las columnas auxiliares
    first_day_per_month = first_day_per_month.drop(columns=['_Year', '_Month'])
    
    # Ordenar por fecha
    first_day_per_month = first_day_per_month.sort_values(date_column).reset_index(drop=True)
    
    # Información del proceso
    if verbose:
        total_records = len(df)
        reduced_records = len(first_day_per_month)
        reduction_pct = ((total_records - reduced_records) / total_records * 100) if total_records > 0 else 0
        print(f"✓ Registros originales: {total_records}")
        print(f"✓ Registros reducidos: {reduced_records}")
        print(f"✓ Reducción: {reduction_pct:.1f}% ({total_records} → {reduced_records} registros)")
    
    return first_day_per_month

def impute_missing_values(df, columns=None, method='forward_fill', date_column='Category', **kwargs):
    """
    Imputa valores faltantes en las columnas especificadas usando diferentes métodos.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame con datos temporales
    columns : list, optional
        Lista de columnas a imputar. Si es None, imputa SOLAR, EOLICA, PCH
    method : str
        Método de imputación. Opciones:
        - 'forward_fill': Forward fill (rellena hacia adelante)
        - 'backward_fill': Backward fill (rellena hacia atrás)
        - 'linear': Interpolación lineal
        - 'polynomial': Interpolación polinomial (grado especificado en kwargs)
        - 'spline': Interpolación spline (orden especificado en kwargs)
        - 'mean': Media de la columna
        - 'median': Mediana de la columna
        - 'knn': K-Nearest Neighbors (n_neighbors especificado en kwargs, default=5)
    date_column : str
        Nombre de la columna de fecha para ordenar los datos (default: 'Category')
    **kwargs
        Argumentos adicionales para métodos específicos:
        - order: para spline (default: 3)
        - degree: para polynomial (default: 2)
        - n_neighbors: para knn (default: 5)
    
    Returns:
    --------
    pd.DataFrame
        DataFrame con valores imputados
    
    Example:
    --------
    >>> df_imputed = impute_missing_values(df, method='linear')
    >>> df_imputed = impute_missing_values(df, method='knn', n_neighbors=3)
    """
    from sklearn.impute import KNNImputer
    
    df_imputed = df.copy()
    
    # Columnas por defecto
    if columns is None:
        columns = ['SOLAR', 'EOLICA', 'PCH']
    
    # Filtrar solo las columnas que existen en el DataFrame
    columns = [col for col in columns if col in df_imputed.columns]
    
    if len(columns) == 0:
        print("⚠ Advertencia: No se encontraron columnas válidas para imputar")
        return df_imputed
    
    # Asegurar que los datos estén ordenados por fecha
    if date_column in df_imputed.columns:
        df_imputed = df_imputed.sort_values(date_column).reset_index(drop=True)
    
    # Contar valores faltantes antes de imputar
    missing_before = df_imputed[columns].isnull().sum()
    total_missing = missing_before.sum()
    
    if total_missing == 0:
        print("✓ No hay valores faltantes para imputar")
        return df_imputed
    
    print(f"Valores faltantes antes de imputar:")
    for col in columns:
        if missing_before[col] > 0:
            print(f"  - {col}: {missing_before[col]} valores faltantes")
    
    # Aplicar método de imputación
    for col in columns:
        if df_imputed[col].isnull().sum() > 0:
            if method == 'forward_fill':
                df_imputed[col] = df_imputed[col].ffill()
            
            elif method == 'backward_fill':
                df_imputed[col] = df_imputed[col].bfill()
            
            elif method == 'linear':
                df_imputed[col] = df_imputed[col].interpolate(method='linear', limit_direction='both')
            
            elif method == 'polynomial':
                degree = kwargs.get('degree', 2)
                df_imputed[col] = df_imputed[col].interpolate(method='polynomial', order=degree, limit_direction='both')
            
            elif method == 'spline':
                order = kwargs.get('order', 3)
                df_imputed[col] = df_imputed[col].interpolate(method='spline', order=order, limit_direction='both')
            
            elif method == 'mean':
                mean_value = df_imputed[col].mean()
                df_imputed[col] = df_imputed[col].fillna(mean_value)
            
            elif method == 'median':
                median_value = df_imputed[col].median()
                df_imputed[col] = df_imputed[col].fillna(median_value)
            
            elif method == 'knn':
                n_neighbors = kwargs.get('n_neighbors', 5)
                # Preparar datos para KNN (solo columnas numéricas)
                numeric_cols = df_imputed.select_dtypes(include=[np.number]).columns.tolist()
                if col in numeric_cols:
                    imputer = KNNImputer(n_neighbors=n_neighbors)
                    # Imputar solo las columnas numéricas
                    df_temp = df_imputed[numeric_cols].copy()
                    df_imputed_values = imputer.fit_transform(df_temp)
                    df_imputed[numeric_cols] = pd.DataFrame(df_imputed_values, columns=numeric_cols, index=df_temp.index)
            
            else:
                print(f"⚠ Método '{method}' no reconocido. Usando forward_fill por defecto.")
                df_imputed[col] = df_imputed[col].ffill()
    
    # Contar valores faltantes después de imputar
    missing_after = df_imputed[columns].isnull().sum()
    total_missing_after = missing_after.sum()
    
    print(f"\n✓ Imputación completada usando método: {method}")
    print(f"✓ Valores faltantes después: {total_missing_after}")
    if total_missing_after > 0:
        print("⚠ Advertencia: Aún quedan valores faltantes después de la imputación")
        for col in columns:
            if missing_after[col] > 0:
                print(f"  - {col}: {missing_after[col]} valores faltantes")
    
    return df_imputed

def compare_imputation_methods(df, columns=None, date_column='Category', methods=None):
    """
    Compara diferentes métodos de imputación y muestra estadísticas.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame con datos temporales
    columns : list, optional
        Lista de columnas a comparar. Si es None, usa SOLAR, EOLICA, PCH
    date_column : str
        Nombre de la columna de fecha
    methods : list, optional
        Lista de métodos a comparar. Si es None, usa métodos por defecto
    
    Returns:
    --------
    dict
        Diccionario con DataFrames imputados para cada método
    """
    if columns is None:
        columns = ['SOLAR', 'EOLICA', 'PCH']
    
    if methods is None:
        methods = ['forward_fill', 'backward_fill', 'linear', 'mean', 'median']
    
    results = {}
    
    print("=" * 80)
    print("COMPARACIÓN DE MÉTODOS DE IMPUTACIÓN")
    print("=" * 80)
    
    for method in methods:
        print(f"\n--- Método: {method} ---")
        df_imputed = impute_missing_values(df.copy(), columns=columns, 
                                          method=method, date_column=date_column)
        results[method] = df_imputed
        
        # Estadísticas después de imputar
        for col in columns:
            if col in df_imputed.columns:
                mean_val = df_imputed[col].mean()
                std_val = df_imputed[col].std()
                print(f"  {col}: Media={mean_val:.2f}, Std={std_val:.2f}")
    
    print("\n" + "=" * 80)
    
    return results


## 3. Funciones de Visualización
    

In [ ]:
def plot_time_series(df, columns=None, figsize=(15, 8), title='Serie Temporal'):
    """
    Grafica series temporales de las columnas especificadas.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame con datos temporales
    columns : list
        Lista de columnas a graficar (default: None, grafica todas las numéricas)
    figsize : tuple
        Tamaño de la figura
    title : str
        Título del gráfico
    """
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns.tolist()
        columns = [col for col in columns if col not in ['Year', 'Month', 'Day', 'DayOfYear']]
    
    fig, ax = plt.subplots(figsize=figsize)
    
    for col in columns:
        if col in df.columns and col != 'Category':
            ax.plot(df['Category'], df[col], label=col, linewidth=2)
    
    ax.set_xlabel('Fecha', fontsize=12)
    ax.set_ylabel('Valor', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


def plot_correlation_matrix(df, columns=None, figsize=(10, 8)):
    """
    Grafica la matriz de correlación.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame con los datos
    columns : list
        Lista de columnas a incluir (default: None, todas las numéricas)
    figsize : tuple
        Tamaño de la figura
    """
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns.tolist()
    
    corr_matrix = df[columns].corr()
    
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
    ax.set_title('Matriz de Correlación', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return corr_matrix


def graph_moving_averages(medias_moviles, datos_originales=None, titulo='Promedios Móviles - Comparación de Ventanas'):
    """
    Genera una matriz de subplots con los promedios móviles.
    
    Parámetros:
    -----------
    medias_moviles : list
        Lista de tuplas (serie, titulo) o solo list de series
        Ejemplo: [(ma_2, 'MA(2)'), (ma_3, 'MA(3)')]
    datos_originales : pd.Series o pd.DataFrame, opcional
        Datos originales para comparar
    """
    n = len(medias_moviles)
    
    # Convertir a formato estándar si es necesario
    if n > 0 and not isinstance(medias_moviles[0], tuple):
        medias_moviles = [(ma, f'Media Móvil {i+1}') for i, ma in enumerate(medias_moviles)]
    
    # Calcular dimensiones
    cols = int(np.ceil(np.sqrt(n)))
    rows = int(np.ceil(n / cols))
    
    fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 5*rows))
    fig.suptitle(titulo, fontsize=16)
    
    # Aplanar axes si es necesario
    if rows == 1:
        axes = [axes] if cols == 1 else axes
    else:
        axes = axes.flatten()
    
    for idx, (ma, titulo) in enumerate(medias_moviles):
        ax = axes[idx]
        
        if datos_originales is not None:
            ax.plot(datos_originales, label='Datos originales', alpha=0.7)
        ax.plot(ma, label=titulo, linewidth=2)
        ax.set_title(titulo)
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    # Ocultar subplots vacíos
    for idx in range(n, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()

## 4. Funciones de Métricas de Evaluación


In [41]:
def calculate_metrics(y_true, y_pred):
    """
    Calcula métricas de evaluación para modelos de regresión.
    
    Parameters:
    -----------
    y_true : array-like
        Valores reales
    y_pred : array-like
        Valores predichos
    
    Returns:
    --------
    dict
        Diccionario con las métricas calculadas
    """
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # MAPE (Mean Absolute Percentage Error)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100
    
    metrics = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'MAPE': mape
    }
    
    return metrics


def print_metrics(metrics):
    """
    Imprime las métricas de evaluación de forma formateada.
    
    Parameters:
    -----------
    metrics : dict
        Diccionario con las métricas
    """
    print("=" * 50)
    print("MÉTRICAS DE EVALUACIÓN")
    print("=" * 50)
    for key, value in metrics.items():
        if key == 'R2':
            print(f"{key:15s}: {value:10.4f}")
        elif key == 'MAPE':
            print(f"{key:15s}: {value:10.2f}%")
        else:
            print(f"{key:15s}: {value:10.4f}")
    print("=" * 50)


## 5. Funciones de Utilidades Generales
    

In [42]:
def get_data_info(df):
    """
    Muestra información resumida del DataFrame.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame a analizar
    """
    print("=" * 60)
    print("INFORMACIÓN DEL DATASET")
    print("=" * 60)
    print(f"Forma del dataset: {df.shape}")
    print(f"Filas: {df.shape[0]}")
    print(f"Columnas: {df.shape[1]}")
    print("\n" + "-" * 60)
    print("TIPOS DE DATOS:")
    print("-" * 60)
    print(df.dtypes)
    print("\n" + "-" * 60)
    print("VALORES FALTANTES:")
    print("-" * 60)
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(missing[missing > 0])
    else:
        print("No hay valores faltantes")
    print("\n" + "-" * 60)
    print("ESTADÍSTICAS DESCRIPTIVAS:")
    print("-" * 60)
    print(df.describe())
    print("=" * 60)


def save_results(df, file_path, sep=';', decimal=','):
    """
    Guarda un DataFrame en formato CSV con el formato del proyecto.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame a guardar
    file_path : str
        Ruta donde guardar el archivo
    sep : str
        Delimitador (default: ';')
    decimal : str
        Separador decimal (default: ',')
    """
    df.to_csv(file_path, sep=sep, decimal=decimal, index=False, encoding='utf-8')
    print(f"Resultados guardados en: {file_path}")


## 6. Ejemplo de Uso

Para usar estas funciones en otros notebooks, puedes importarlas así:
    
```python
# Opción 1: Ejecutar este notebook completo y luego importar
%run ../utils/funcions.ipynb

# Opción 2: Convertir a módulo Python y luego importar
# from utils.funcions import load_capacidad_data, plot_time_series, calculate_metrics
```


## 7. Ejemplo Práctico

A continuación se muestra un ejemplo de cómo usar las funciones:


In [43]:
# Ejemplo de uso (descomentar para ejecutar)
# 
# # 1. Cargar datos
# df = load_capacidad_data('../data/capacidad_acumulada.csv')
# print(f"Registros originales: {len(df)}")
# 
# # 2. Extraer primer día de cada mes (reducir datos)
# df_reduced = extract_first_day_per_month(df, date_column='Category')
# print(f"Registros reducidos: {len(df_reduced)}")
# 
# # 3. Preprocesar
# df_processed = preprocess_capacidad_data(df_reduced)
# 
# # 4. Ver información
# get_data_info(df_processed)
# 
# # 5. Visualizar series temporales
# plot_time_series(df_processed, columns=['SOLAR', 'EOLICA', 'PCH'], 
#                  title='Capacidad Acumulada de Energía Renovable')
# 
# # 6. Matriz de correlación
# corr_matrix = plot_correlation_matrix(df_processed, columns=['SOLAR', 'EOLICA', 'PCH'])
# 
# # 7. Guardar resultados si es necesario
# # save_results(df_reduced, '../data/capacidad_acumulada2.csv')
